# Demos for Talk Working with Large Models in ONNX IR

In [ ]:
# Prepare environment

%pip install --upgrade onnxscript onnx-ir onnx-safetensors model-explorer-onnx onnxruntime

## Demo 1: Safetensors in ONNX

**Q1: Is there a way to use the safetensors format as an external data format for ONNX?**

**A1:** Yes. The data is contiguous, row-major, and little-endian (same as ONNX). Data offset can be found by parsing the json header.

<img src="resources/safetensors-format.svg" width="500"/>

Image source: https://huggingface.co/docs/safetensors/en/index

**Q2: How do we do it efficiently?**

**A2:** Use onnx_ir to replace the tensors.

In [26]:
import onnx_ir as ir

model = ir.load("resources/model.textproto")
print(model)

<
    ir_version=10,
    opset_imports={'': 21},
    producer_name='onnx-safetensors-example',
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=SimpleGraph,
    inputs=(
        %"input"<FLOAT,[1,3]>
    ),
    outputs=(
        %"output"<FLOAT,[1,3]>
    ),
    initializers=(
        %"weights"<FLOAT,[3]>{TensorProtoTensor<FLOAT,[3]>(array([1., 2., 3.], dtype=float32), name='weights')}
    ),
) {
    0 |  # :anonymous_node:130470381133344
         %"output"<FLOAT,[1,3]> ⬅️ ::Add(%"input", %"weights"{[1.0, 2.0, 3.0]})
    return %"output"<FLOAT,[1,3]>
}




### Loading tensors from a safetensors file into an ONNX model

Use `load_file_as_external_data` to load safetensors as external data and replace weights in the model

In [25]:
model_with_external_data = onnx_safetensors.load_file_as_external_data(
    model, "resources/weights.safetensors"  # weights containing [4, 5, 6]
)

print(onnx.printer.to_text(ir.to_proto(model_with_external_data)))

<
   ir_version: 10,
   opset_import: ["" : 21],
   producer_name: "onnx-safetensors-example"
>
SimpleGraph (float[1,3] input) => (float[1,3] output) 
   <float[3] weights = ["location": "resources/weights.safetensors", "offset": "72", "length": "12"], float[3] weights>
{
   output = Add (input, weights)
}


### Using safetensors as external data for ONNX

We can similarly save external data file from an ONNX model to safetensors. By storing the tensor dtype in ONNX file, we can even use types safetensors doesn't yet support, like INT4.

You can read more at https://github.com/justinchuby/onnx-safetensors/blob/main/examples/tutorial.ipynb

### Inference with ONNX Runtime

In [23]:
import onnxruntime as ort

ir.save(model_with_external_data, "model_with_external_data.onnx")
session = ort.InferenceSession("model_with_external_data.onnx")
output = session.run(None, {"input": np.array([[1.0, 2.0, 3.0]], dtype=np.float32)})
print("[[1.0, 2.0, 3.0]] + [4, 5, 6] =", output)

[[1.0, 2.0, 3.0]] + [4, 5, 6] = [array([[5., 7., 9.]], dtype=float32)]


## Demo 2: Fusion in onnxscript for ONNX Runtime

## Demo 3: Putting it together

1. Build a model with the tape module.
2. Load a model and replace some initializers
3. Build a pass to modify the model (merge QKV weights)
4. Use rewriter to do the same thing
5. Show it on model explorer


In [ ]:
from onnx_ir import tape

ir.Value(name="input", type=ir.TensorType(ir.DataType.FLOAT), shape=ir.Shape(["N", 3]))
ir.Value(name="q_weight")